In [16]:
import cv2
import matplotlib.pyplot as plt
from math import floor

minAR = 1.7
maxAR = 6.1
minArea = 300
maxArea = 29000

for i in range(1, 51):
    filename = "../../data/Vehicles/{:04d}.jpg".format(i)

    image = cv2.imread(filename)
    image_ar = image.shape[0] / image.shape[1]
    scale_factor = 500 / image.shape[1]
    image = cv2.resize(image, (0, 0), fx=scale_factor, fy=scale_factor)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    bilateral = cv2.bilateralFilter(gray, 13, 17, 17)

    edges = cv2.Canny(bilateral, 40, 200)
    # plt.figure(figsize=[15, 15])
    # plt.imshow(edges, cmap="gray")

    # edges = cv2.GaussianBlur(edges, (7, 7), 0)
    # plt.figure(figsize=[15, 15])
    # plt.imshow(edges, cmap="gray")

    edges = cv2.morphologyEx(
        edges,
        cv2.MORPH_CLOSE,
        cv2.getStructuringElement(
            cv2.MORPH_RECT, (round(17 * image_ar), round(7 * image_ar))
        ),
    )
    # plt.figure(figsize=[15, 15])
    # plt.imshow(edges, cmap="gray")

    # thresh = cv2.threshold(edges, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

    # plt.figure(figsize=[15, 15])
    # plt.imshow(thresh, cmap="gray")

    # thresh = cv2.dilate(thresh, cv2.getStructuringElement(cv2.MORPH_RECT, (7, 2)))
    # thresh = cv2.erode(thresh, cv2.getStructuringElement(cv2.MORPH_RECT, (3, 2)))

    # plt.figure(figsize=[15, 15])
    # plt.imshow(thresh, cmap="gray")

    radius = 15
    disk_kernel = cv2.getStructuringElement(
        cv2.MORPH_ELLIPSE, (2 * radius + 1, 2 * radius + 1), (radius, radius)
    )
    opened_image = cv2.morphologyEx(gray, cv2.MORPH_OPEN, disk_kernel)

    subtracted_image = cv2.subtract(gray, opened_image)

    """ perform a blackhat morphological operation to reveal dark characters (letters, digits, and symbols)
    against light backgrounds (the license plate itself)"""
    rectKern = cv2.getStructuringElement(cv2.MORPH_RECT, (13, 7))
    blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, rectKern)

    """ find regions in the image that are light and may contain license plate characters
    find regions in the image that are light and may contain license plate characters """
    squareKern = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    light = cv2.morphologyEx(subtracted_image, cv2.MORPH_CLOSE, squareKern)
    light = cv2.threshold(light, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

    thresh = cv2.bitwise_and(subtracted_image, subtracted_image, mask=light)
    thresh = cv2.dilate(
        thresh,
        cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3)),
        iterations=2,
    )
    thresh = cv2.erode(
        thresh, cv2.getStructuringElement(cv2.MORPH_RECT, (13, 6)), iterations=2
    )

    thresh = cv2.GaussianBlur(thresh, (5, 5), 0)
    thresh = cv2.dilate(
        thresh, cv2.getStructuringElement(cv2.MORPH_RECT, (2, 1)), iterations=1
    )
    # plt.figure(figsize=[15, 15])
    # plt.imshow(thresh, cmap="gray")

    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(
        contours,
        key=cv2.contourArea,
        reverse=True,
    )[:20]

    # figure = plt.figure()
    # figure.add_subplot(1, 1, 1)
    # cv2.drawContours(image, contours, -1, (0, 255, 0), 2)

    # plt.figure(figsize=[15, 15])
    # plt.imshow(image)
    licensePlate = None
    for c in contours:
        (x, y, w, h) = cv2.boundingRect(c)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        ar = w / float(h)
        area = cv2.contourArea(c)
        area_ratio = area / (w * h)

        if (
            minAR <= ar <= maxAR
            and minArea <= w * h <= maxArea
            and w>60 and h>20

        ):
            cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
            licensePlate = gray[y : y + h, x : x + w]
    if licensePlate is None:
        licensePlate = gray
    else:
        cv2.imwrite(
            "../../data/Testing_output/{:04d}[00].jpg".format(i),
            licensePlate if not None else image,
        )
    # plt.figure(figsize=[15, 15])
    # plt.imshow(image)

    # # Draw license plate
    # plt.figure(figsize=[15, 15])
    # plt.imshow(licensePlate, cmap="gray")